In [1]:
import os
import pandas as pd
import nltk
import gensim
import numpy as np
from gensim import corpora, models, similarities

In [2]:
df_pos_onclinic = pd.read_json('../onclinic.json')
df_neg_vibokep = pd.read_json('../vibokep.json')
df_neg_xpanas = pd.read_json('../xpanas.json')
df_neg_bokep2017 = pd.read_json('../bokep2017.json')
df_neg_aora = pd.read_json('../aora.json')
df_neg_bbindo = pd.read_json('../bbindo.json')

In [3]:
corpus_1 = df_neg_vibokep['title'].values.tolist()
corpus_2 = df_neg_xpanas['title'].values.tolist()
corpus_3 = df_neg_bokep2017['title'].values.tolist()
corpus_4 = df_neg_aora['title'].values.tolist()
corpus_5 = df_neg_bbindo['title'].values.tolist()
corpus_6 = df_pos_onclinic['content'].values.tolist()

In [4]:
corpus = corpus_1 + corpus_2 + corpus_3 + corpus_4 + corpus_5 + corpus_6

In [5]:
tok_corp = [nltk.word_tokenize(sent) for sent in corpus]

In [6]:
model = gensim.models.Word2Vec(tok_corp,min_count=1,size=32)

In [11]:
model.most_similar('saya')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('On', 0.9473282098770142),
 ('(', 0.9457900524139404),
 ('seks', 0.9453378915786743),
 ('Bokep', 0.944746732711792),
 ('Indonesia', 0.9423506259918213),
 ('dia', 0.9419797658920288),
 ('ngentot', 0.9417904615402222),
 ('atau', 0.9414888620376587),
 ('melakukan', 0.9409876465797424),
 ('seksual', 0.9402721524238586)]

In [8]:
model.wv.doesnt_match("pria dan wanita makan bersama".split())

'makan'

In [13]:
model.wv.syn0.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


(4196, 32)

In [15]:
# Function to average all word vectors in a paragraph
def featureVecMethod(words, model, num_features):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    
    #Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model.wv.index2word)
    
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    

    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

In [16]:
# Function for calculating the average feature vector
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        # Printing a status message every 1000th review
        if counter%1000 == 0:
            print("Review %d of %d"%(counter,len(reviews)))
            
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
        
    return reviewFeatureVecs

In [17]:
# Calculating average feature vector for training set
clean_train_reviews = []
for review in train['review']:
    clean_train_reviews.append(review_wordlist(review, remove_stopwords=True))
    
trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)


NameError: name 'train' is not defined

In [18]:
# Fitting a random forest classifier to the training data
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)
    
print("Fitting random forest to training data....")    
forest = forest.fit(trainDataVecs, train["sentiment"])

Fitting random forest to training data....


NameError: name 'trainDataVecs' is not defined

In [14]:
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

In [9]:
def visualize(model, output_path):
    meta_file = "w2x_metadata.tsv"
    placeholder = np.zeros((len(model.wv.index2word), 32))

    with open(os.path.join(output_path,meta_file), 'wb') as file_metadata:
        for i, word in enumerate(model.wv.index2word):
            placeholder[i] = model[word]
            # temporary solution for https://github.com/tensorflow/tensorflow/issues/9094
            if word == '':
                print("Emply Line, should replecaed by any thing else, or will cause a bug of tensorboard")
                file_metadata.write("{0}".format('<Empty Line>').encode('utf-8') + b'\n')
            else:
                file_metadata.write("{0}".format(word).encode('utf-8') + b'\n')

    # define the model without training
    sess = tf.InteractiveSession()

    embedding = tf.Variable(placeholder, trainable = False, name = 'w2x_metadata')
    tf.global_variables_initializer().run()

    saver = tf.train.Saver()
    writer = tf.summary.FileWriter(output_path, sess.graph)

    # adding into projector
    config = projector.ProjectorConfig()
    embed = config.embeddings.add()
    embed.tensor_name = 'w2x_metadata'
    embed.metadata_path = meta_file

    # Specify the width and height of a single thumbnail.
    projector.visualize_embeddings(writer, config)
    saver.save(sess, os.path.join(output_path,'w2x_metadata.ckpt'))
    print('Run `tensorboard --logdir={0}` to run visualize result on tensorboard'.format(output_path))

In [11]:
# visualize(model,"./")

c:\users\devan\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


Run `tensorboard --logdir=./` to run visualize result on tensorboard


In [12]:
model.save('model_cnn_edition')